In [ ]:
import pandas as pd



Index(['date', 'open_price', 'high_price', 'low_price', 'close_price',
       'volume', 'adjclose_price'],
      dtype='object')


Horizontal X = ordinalDate
Vertikal Y = close pris